# Sample Script
This is a sample script that will run in the semi custom framework.

**Script**
* [scripts/custom/sample_script.ipynb](./scripts/custom/sample_script.ipynb)

**Parameters**
* `in/custom/sample_input.xlsx[param]`

**Input**
* `sample_ref_table`: `in/custom/sample_input.xlsx[ref]`

**Output**
* TBD

In [1]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/connect/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications

# Parameters
Import analysis parameters from an excel spreadsheet

**Input**  
* `in/custom/sample_input.xlsx[param]`

**Output**
* Python variables named after parameters with the value


In [2]:
#Create system variables from excel into script and review values in dictionary
input_df = pd.read_excel('../../in/custom/sample_input.xlsx', sheet_name='param', skiprows=4, dtype=str)
var_dict = dict(zip(input_df.parameter, input_df.value))
for key,val in var_dict.items(): exec(key + '=val')

#Check inputs
pd.DataFrame.from_dict(var_dict, orient='index')

,0
sample_date,2018-12-31
sample_date2,2017-12-31


# Reference
Upload a reference table from excel to snowflake

**Input**  
* `in/custom/sample_input.xlsx[ref]`

**Output**  
* `sample_ref_table`

In [5]:
df

,Animal,breed
0,Dog,golden retriever
1,Dog,mini golden doodle
2,Dog,sheltie


In [6]:
#Upload reference table from excel to snowflake and review snowflake output
df = pd.read_excel('../../in/custom/sample_input.xlsx', sheet_name='ref', skiprows=4, dtype=str)

#Strip white space and make referrable columns uppercase
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df[['animal','breed']] =  \
   df[['animal','breed']].apply(lambda x: x.str.upper() if x.dtype == "object" else x)

#Upload to snowflake
snow.drop_table("sample_ref_table")
snow.upload_dataframe(df,"sample_ref_table")
snow.select("SELECT * FROM sample_ref_table")

DROP TABLE IF EXISTS ref_db.semi_custom.sample_ref_table;
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Table ref_db.semi_custom.sample_ref_table dropped! (╯°□°）╯︵ ┻━┻
Upload into ref_db.semi_custom.sample_ref_table successful! ┬──┬◡ﾉ(°-°ﾉ)


,animal,breed
0,DOG,GOLDEN RETRIEVER
1,DOG,MINI GOLDEN DOODLE
2,DOG,SHELTIE
